# Make MTC base network

In this notebook we will run through:

1. Read in wrangler standard network  
2. calculate NUMLANES, append centroid and centroid connectors
3. Transforming the standard network format to the MTC expected format    
4. Exporting the network to shapefile, CUBE, network standard, or pickle

In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import MTC

import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

# Remote I/O

In [4]:
input_dir = "../../travel-model-two-networks/data/interim/step8_standard_format"
output_dir = "../tests/scratch"

In [5]:
parameters = Parameters()

2020-10-21 12:26:12, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso
2020-10-21 12:26:12, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso


In [6]:
parameters.int_col

['model_link_id',
 'model_node_id',
 'A',
 'B',
 'drive_access',
 'walk_access',
 'bike_access',
 'truck_access',
 'ML_lanes_AM',
 'ML_lanes_MD',
 'ML_lanes_PM',
 'ML_lanes_EV',
 'ML_lanes_EA',
 'segment_id',
 'managed',
 'bus_only',
 'rail_only',
 'ft',
 'lanes_AM',
 'lanes_MD',
 'lanes_PM',
 'lanes_EA',
 'lanes_EV',
 'useclass_AM',
 'useclass_EA',
 'useclass_MD',
 'useclass_PM',
 'useclass_EV',
 'tollseg',
 'tollbooth']

In [7]:
parameters.properties_to_split

{'lanes': {'v': 'lanes',
  'time_periods': {'EA': ('3:00', '6:00'),
   'AM': ('6:00', '10:00'),
   'MD': ('10:00', '15:00'),
   'PM': ('15:00', '19:00'),
   'EV': ('19:00', '3:00')}},
 'ML_lanes': {'v': 'ML_lanes',
  'time_periods': {'EA': ('3:00', '6:00'),
   'AM': ('6:00', '10:00'),
   'MD': ('10:00', '15:00'),
   'PM': ('15:00', '19:00'),
   'EV': ('19:00', '3:00')}},
 'useclass': {'v': 'useclass',
  'time_periods': {'EA': ('3:00', '6:00'),
   'AM': ('6:00', '10:00'),
   'MD': ('10:00', '15:00'),
   'PM': ('15:00', '19:00'),
   'EV': ('19:00', '3:00')}}}

# Read Roadway and Transit Networks

In [8]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(
    link_file = link_file, 
    node_file = node_file, 
    shape_file = shape_file
)

2020-10-21 12:26:15, INFO: Reading from following files:
-../../travel-model-two-networks/data/interim/step8_standard_format\link.json
-../../travel-model-two-networks/data/interim/step8_standard_format\node.geojson
-../../travel-model-two-networks/data/interim/step8_standard_format\shape.geojson.
2020-10-21 12:26:15, INFO: Reading from following files:
-../../travel-model-two-networks/data/interim/step8_standard_format\link.json
-../../travel-model-two-networks/data/interim/step8_standard_format\node.geojson
-../../travel-model-two-networks/data/interim/step8_standard_format\shape.geojson.
2020-10-21 12:29:23, INFO: Read 1634769 links from ../../travel-model-two-networks/data/interim/step8_standard_format\link.json
2020-10-21 12:29:23, INFO: Read 1634769 links from ../../travel-model-two-networks/data/interim/step8_standard_format\link.json
2020-10-21 12:29:23, INFO: Read 644475 nodes from ../../travel-model-two-networks/data/interim/step8_standard_format\node.geojson
2020-10-21 12:29

In [9]:
roadway_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry'],
      dtype='object')

In [10]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2020-10-21 12:33:50, INFO: Read in transit feed from: ../../travel-model-two-networks/data/interim/step8_standard_format
2020-10-21 12:33:50, INFO: Read in transit feed from: ../../travel-model-two-networks/data/interim/step8_standard_format
2020-10-21 12:33:50, DEBUG: ...agency.txt:
  agency_id                    agency_name  \
0      BART         Bay Area Rapid Transit   
1        CE  ACE Altamont Corridor Express   
2       NaN              County Connection   
3       573                 Emery Go-Round   
4        36   Fairfield and Suisun Transit   
5       NaN            Golden Gate Transit   
6       533             MVgo Mountain View   
7       NaN    Stanford Marguerite Shuttle   
8      MCTD                  Marin Transit   
9         5         Rio Vista Delta Breeze   

                             agency_url      agency_timezone agency_lang  \
0                   http://www.bart.gov  America/Los_Angeles          en   
1               http://www.acerail.com/  America/Los_Ang

2020-10-21 12:33:50, DEBUG: ...calendar_dates.txt:
Empty DataFrame
Columns: [service_id, date, exception_type]
Index: []
2020-10-21 12:33:50, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-10-21 12:33:50, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-10-21 12:33:52, DEBUG: ...fare_attributes.txt:
Empty DataFrame
Columns: [fare_id, price, currency_type, payment_method, transfers]
Index: []
2020-10-21 12:33:52, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-10-21 12:33:52, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-10-21 12:33:52, DEBUG: ...fare_rules.txt:
Empty DataFrame
Columns: [fare_id]
Index: []
2020-10-21 12:33:52, INFO: Removing fare_rules.txt from transit network config because file not found
2020-10-21 12:33:52, INFO: Removing fare_rules.txt from transit network config because file not found
2020-10-21 

2020-10-21 12:33:53, DEBUG: ...transfers.txt:
Empty DataFrame
Columns: [from_stop_id, to_stop_id, transfer_type]
Index: []
2020-10-21 12:33:53, INFO: Removing transfers.txt from transit network config because file not found
2020-10-21 12:33:53, INFO: Removing transfers.txt from transit network config because file not found
2020-10-21 12:33:53, DEBUG: ...feed_info.txt:
Empty DataFrame
Columns: [feed_publisher_name, feed_publisher_url, feed_lang]
Index: []
2020-10-21 12:33:53, INFO: Removing feed_info.txt from transit network config because file not found
2020-10-21 12:33:53, INFO: Removing feed_info.txt from transit network config because file not found
2020-10-21 12:33:53, WARNING: The following stop_ids from are referenced but missing from stops.txt: ['place_PAHE', 'place_WOBA', 'place_4SSP', 'ctsb', 'place_11BD', 'place_CSSH', 'place_11JE', 'place_HPNR', 'place_14BD', 'ctla', 'place_CCCO', 'place_CVBA', 'ctsmar', 'place_90MA', 'ct22', 'ctcap', 'place_FVMA', 'ctmv', 'place_NAMO', 'pla

# Make Travel Model Network

In [11]:
base_roadway_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = roadway_net, 
    parameters = parameters)

2020-10-21 12:38:23, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso
2020-10-21 12:38:23, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso


In [12]:
print(base_roadway_net.links_df.shape)
print(base_roadway_net.shapes_df.shape)
print(base_roadway_net.nodes_df.shape)

(1634769, 26)
(869970, 7)
(644475, 11)


In [13]:
base_roadway_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry'],
      dtype='object')

In [14]:
roadway_net.nodes_df.head(3)

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry
model_node_id_idx,,,,,,,,,,,
3000000,2.401245e+09,505d64eb98f1da8d812a3b3801034308,Contra Costa,1,1,1,3000000,0,-122.331554,37.981204,POINT (-122.33155 37.98120)
3000001,5.783907e+07,473979c78435732f01ca5a168afb62e0,Contra Costa,1,1,1,3000001,0,-121.944770,37.953322,POINT (-121.94477 37.95332)
5000000,1.024389e+09,fc7b575d5d8c961d4a70fca846ae7f80,Marin,1,1,1,5000000,0,-122.539828,37.897999,POINT (-122.53983 37.89800)


## Add centroid and centroid connector database

In [15]:
base_roadway_net = MTC.add_centroid_and_centroid_connector(
    roadway_network_object = base_roadway_net
)

2020-10-21 12:38:24, INFO: Adding centroid and centroid connector to standard network
2020-10-21 12:38:24, INFO: Adding centroid and centroid connector to standard network
2020-10-21 12:38:41, INFO: Finished adding centroid and centroid connectors
2020-10-21 12:38:41, INFO: Finished adding centroid and centroid connectors


In [16]:
type(base_roadway_net.parameters)

lasso.parameters.Parameters

In [17]:
print(base_roadway_net.links_df.shape)
print(base_roadway_net.shapes_df.shape)
print(base_roadway_net.nodes_df.shape)

(2221035, 26)
(1163103, 7)
(688952, 11)


In [18]:
base_roadway_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry'],
      dtype='object')

In [19]:
base_roadway_net.nodes_df.columns

Index(['osm_node_id', 'shst_node_id', 'county', 'drive_access', 'walk_access',
       'bike_access', 'model_node_id', 'rail_only', 'X', 'Y', 'geometry'],
      dtype='object')

## Calculate number of lanes

In [20]:
base_roadway_net = MTC.determine_number_of_lanes(
    roadway_network_object = base_roadway_net,
    network_variable = 'lanes'
)

base_roadway_net.links_df.lanes.value_counts()

2020-10-21 12:38:41, INFO: Determining number of lanes
2020-10-21 12:38:41, INFO: Determining number of lanes
2020-10-21 12:45:01, INFO: Finished determining number of lanes using variable: lanes
2020-10-21 12:45:01, INFO: Finished determining number of lanes using variable: lanes


1    2072663
2     104207
3      22455
4      14124
5       6504
6        929
7        134
8         18
9          1
Name: lanes, dtype: int64

In [21]:
pd.crosstab(base_roadway_net.links_df.roadway, base_roadway_net.links_df.lanes)

lanes,1,2,3,4,5,6,7,8,9
roadway,,,,,,,,,
cycleway,99067,36,4,0,0,0,0,0,0
footway,247365,87,27,8,2,0,0,0,0
maz,531308,0,0,0,0,0,0,0,0
motorway,115,576,640,1026,314,96,6,1,0
motorway_link,3561,1184,187,44,7,0,0,0,0
primary,2555,11633,7466,3151,1367,235,71,14,0
primary_link,1356,110,35,3,2,0,0,0,0
residential,503743,33033,1424,369,90,7,0,0,0
secondary,25600,25120,7300,6494,3534,435,44,2,0


## MTC format

In [22]:
base_roadway_net.parameters.int_col

['model_link_id',
 'model_node_id',
 'A',
 'B',
 'drive_access',
 'walk_access',
 'bike_access',
 'truck_access',
 'ML_numlanes_AM',
 'ML_numlanes_MD',
 'ML_numlanes_PM',
 'ML_numlanes_EV',
 'ML_numlanes_EA',
 'segment_id',
 'managed',
 'bus_only',
 'rail_only',
 'ft',
 'numlanes_AM',
 'numlanes_MD',
 'numlanes_PM',
 'numlanes_EA',
 'numlanes_EV',
 'useclass_AM',
 'useclass_EA',
 'useclass_MD',
 'useclass_PM',
 'useclass_EV',
 'use',
 'tollseg',
 'tollbooth']

In [22]:
base_roadway_net.roadway_standard_to_mtc_network()

2020-10-19 17:05:21, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-10-19 17:05:21, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-10-19 17:05:21, INFO: Didn't detect managed lanes in network.
2020-10-19 17:05:21, INFO: Didn't detect managed lanes in network.
2020-10-19 17:05:21, INFO: Creating calculated roadway variables.
2020-10-19 17:05:21, INFO: Creating calculated roadway variables.
2020-10-19 17:05:21, INFO: Calculating Facility Type
2020-10-19 17:05:21, INFO: Calculating Facility Type
2020-10-19 17:08:29, INFO: Finished calculating roadway class variable: ft
2020-10-19 17:08:29, INFO: Finished calculating roadway class variable: ft
2020-10-19 17:08:29, INFO: Determining assignable
2020-10-19 17:08:29, INFO: Determining assignable
2020-10-19 17:08:29, INFO: Calculating and adding roadway network variable: assignable
2020-10-19 17:08:29, INFO: Calculating and adding roadway network variable: a

# Write Out

## As shapefile

In [40]:
base_roadway_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2020-10-19 13:37:20, INFO: Writing Network as Shapefile
2020-10-19 13:37:20, DEBUG: Output Variables: 
 - model_link_id
 - link_id
 - A
 - B
 - shstGeometryId
 - distance
 - roadway
 - name
 - bike_access
 - walk_access
 - drive_access
 - truck_access
 - numlanes_EA
 - numlanes_AM
 - numlanes_MD
 - numlanes_PM
 - numlanes_EV
 - county
 - model_node_id
 - N
 - osm_node_id
 - geometry
 - X
 - Y
 - ML_lanes_EA
 - ML_lanes_AM
 - ML_lanes_MD
 - ML_lanes_PM
 - ML_lanes_EV
 - segment_id
 - managed
 - bus_only
 - rail_only
 - assignable
 - cntype
 - useclass
 - transit
 - tollbooth
 - tollseg
 - ft
 - tap_drive
 - use
 - tollbooth
 - tollseg
2020-10-19 13:37:20, DEBUG: Network Link Variables: 
 - access
 - bike_access
 - drive_access
 - fromIntersectionId
 - lanes
 - maxspeed
 - name
 - oneWay
 - ref
 - roadway
 - shstGeometryId
 - shstReferenceId
 - toIntersectionId
 - u
 - v
 - walk_access
 - wayId
 - county
 - model_link_id
 - A
 - B
 - rail_traveltime
 - rail_only
 - locationReferences
 - 

## As fixed width (for CUBE)

In [42]:
base_roadway_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
)

2020-10-19 14:06:56, DEBUG: Network Link Variables: 
 - access
 - bike_access
 - drive_access
 - fromIntersectionId
 - lanes
 - maxspeed
 - name
 - oneWay
 - ref
 - roadway
 - shstGeometryId
 - shstReferenceId
 - toIntersectionId
 - u
 - v
 - walk_access
 - wayId
 - county
 - model_link_id
 - A
 - B
 - rail_traveltime
 - rail_only
 - locationReferences
 - shape_id
 - geometry
 - numlanes
 - ft
 - assignable
 - cntype
 - transit
 - useclass
 - use
 - ML_lanes
 - distance
 - numlanes_EA
 - numlanes_AM
 - numlanes_MD
 - numlanes_PM
 - numlanes_EV
 - ML_lanes_EA
 - ML_lanes_AM
 - ML_lanes_MD
 - ML_lanes_PM
 - ML_lanes_EV
2020-10-19 14:06:56, DEBUG: Network Node Variables: 
 - osm_node_id
 - shst_node_id
 - county
 - drive_access
 - walk_access
 - bike_access
 - N
 - rail_only
 - X
 - Y
 - geometry
 - x
 - y
2020-10-19 14:06:57, INFO: Starting fixed width convertion
2020-10-19 14:28:52, INFO: Writing out link database
2020-10-19 14:30:58, INFO: Writing out link header and width ----
2020-10

standard_transit_net = StandardTransit.fromTransitNetwork(transit_net)
standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, "transit.lin"))

## As network standard

base_roadway_net.write(path = output_dir, filename = 'v01')

## As pickle

In [22]:
roadway_filename = os.path.join(output_dir, 'base_roadway_network.pickle')
pickle.dump(base_roadway_net, open(roadway_filename, 'wb'))